# Assignment 3 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {"OH": "Ohio", "KY": "Kentucky", "AS": "American Samoa", "NV": "Nevada", "WY": "Wyoming", "NA": "National", "AL": "Alabama", "MD": "Maryland", "AK": "Alaska", "UT": "Utah", "OR": "Oregon", "MT": "Montana", "IL": "Illinois", "TN": "Tennessee", "DC": "District of Columbia", "VT": "Vermont", "ID": "Idaho", "AR": "Arkansas", "ME": "Maine", "WA": "Washington", "HI": "Hawaii", "WI": "Wisconsin", "MI": "Michigan", "IN": "Indiana", "NJ": "New Jersey", "AZ": "Arizona", "GU": "Guam", "MS": "Mississippi", "PR": "Puerto Rico", "NC": "North Carolina", "TX": "Texas", "SD": "South Dakota", "MP": "Northern Mariana Islands", "IA": "Iowa", "MO": "Missouri", "CT": "Connecticut", "WV": "West Virginia", "SC": "South Carolina", "LA": "Louisiana", "KS": "Kansas", "NY": "New York", "NE": "Nebraska", "OK": "Oklahoma", "FL": "Florida", "CA": "California", "CO": "Colorado", "PA": "Pennsylvania", "DE": "Delaware", "NM": "New Mexico", "RI": "Rhode Island", "MN": "Minnesota", "VI": "Virgin Islands", "NH": "New Hampshire", "MA": "Massachusetts", "GA": "Georgia", "ND": "North Dakota", "VA": "Virginia"}

In [3]:
def get_list_of_university_towns():
    """Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character "\n". """
    
    # Load
    u = pd.read_table("../data/university_towns.txt", sep="\n") # u = pd.read_table("university_towns.txt")
    
    # Change name column and add column_name
    u = u.rename(columns={"Alabama[edit]": "Name"})
    u2 = pd.DataFrame([{"Name": "Alabama[edit]"}])
    u = u2.append(u)
    
    lista1 = []
    lista2 = []
    item = [col for col in u["Name"]]
    for i in range(0,len(item)):
        region = item[i]
        if item[i].find("[ed") != -1: # If characters "[ed" are not in item[i], method .fin() == -1
            state = item[i].split("[ed")[0]
            region = state
        if item[i].find(" (") != -1:
            region = item[i].split(" (")[0]

        lista1.append(state)
        lista2.append(region)

    df = pd.DataFrame([lista1, lista2], index=["State", "RegionName"])
    df = df.T
    df = df[(df["State"] != df["RegionName"])]
    df = df.reset_index()
    del df["index"]
    
    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [4]:
def get_recession_start():
    """Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3"""
    
    # Load and clean
    gdp = pd.read_excel("../data/gdplev.xls", skiprows=5)
    gdp = gdp.rename(columns={"Unnamed: 0": "Year", 
                              "Unnamed: 4": "Year q", 
                              "GDP in billions of current dollars.1": "GDP current", 
                              "GDP in billions of chained 2009 dollars.1": "GDP chained 2009"})
    gdp.drop(labels=["Unnamed: 3", "Unnamed: 7"], axis = 1, inplace = True)
    columns = ["Year q", 
               "GDP current", 
               "GDP chained 2009"]
    gdp = gdp[columns]
    gdp = gdp.dropna()
    gdp = gdp.reset_index()
    del gdp["index"]
    
    # Now calculate year recession
    
    x = []

    # gdp[(gdp["Year q"] == "2000q1")].index -> we need the index after year 2000
    # .index give us "int64index" -> this is a list even contains 1 element
    # So we use .index[0] which is a int()

    for i in range(gdp[(gdp["Year q"] == "2000q1")].index[0],len(gdp)-2): # Start at recession after year 2000
        if (gdp.loc[i]["GDP chained 2009"] > gdp.loc[i+1]["GDP chained 2009"] and gdp.loc[i+1]["GDP chained 2009"] > gdp.loc[i+2]["GDP chained 2009"]):
            x.append(gdp.loc[i]["Year q"])
    
    return x[1] # Always is the second element

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    """Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3"""
    
    # Load and clean
    gdp = pd.read_excel("../data/gdplev.xls", skiprows=5)
    gdp = gdp.rename(columns={"Unnamed: 0": "Year", 
                              "Unnamed: 4": "Year q", 
                              "GDP in billions of current dollars.1": "GDP current", 
                              "GDP in billions of chained 2009 dollars.1": "GDP chained 2009"})
    gdp.drop(labels=["Unnamed: 3", "Unnamed: 7"], axis = 1, inplace = True)
    columns = ["Year q", 
               "GDP current", 
               "GDP chained 2009"]
    gdp = gdp[columns]
    gdp = gdp.dropna()
    gdp = gdp.reset_index()
    del gdp["index"]
    
    # Call function recession_start
    
    x = get_recession_start()
    
    y = []

    # gdp[(gdp["Year q"] == x)].index -> we need the index after recession start
    # .index give us "int64index" -> this is a list even contains 1 element
    # So we use .index[0] which is a int()

    for i in range(gdp[(gdp["Year q"] == x)].index[0],len(gdp)-2): # Recession start at x
        if (gdp.loc[i]["GDP chained 2009"] < gdp.loc[i+1]["GDP chained 2009"] and gdp.loc[i+1]["GDP chained 2009"] < gdp.loc[i+2]["GDP chained 2009"]):
            y.append(gdp.loc[i+1]["Year q"])
       
    return y[1]

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    """Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3"""
    
    # Load and clean
    gdp = pd.read_excel("../data/gdplev.xls", skiprows=5)
    gdp = gdp.rename(columns={"Unnamed: 0": "Year", 
                              "Unnamed: 4": "Year q", 
                              "GDP in billions of current dollars.1": "GDP current", 
                              "GDP in billions of chained 2009 dollars.1": "GDP chained 2009"})
    gdp.drop(labels=["Unnamed: 3", "Unnamed: 7"], axis = 1, inplace = True)
    columns = ["Year q", 
               "GDP current", 
               "GDP chained 2009"]
    gdp = gdp[columns]
    gdp = gdp.dropna()
    gdp = gdp.reset_index()
    del gdp["index"]
    
    # Get the recession interval
    x = get_recession_start()
    y = get_recession_end()
    
    # Dataframe with the recession interval
    p = gdp.loc[gdp[(gdp["Year q"] == x)].index[0]:gdp[(gdp["Year q"] == y)].index[0]]
    print(p.reset_index())
    return p.reset_index().loc[p["GDP chained 2009"].argmin()]["Year q"]

get_recession_bottom()

   index  Year q  GDP current  GDP chained 2009
0    246  2008q3      14843.0           14891.6
1    247  2008q4      14549.9           14577.0
2    248  2009q1      14383.9           14375.0
3    249  2009q2      14340.4           14355.6
4    250  2009q3      14384.1           14402.5
5    251  2009q4      14566.5           14541.9


'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    """Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    """
    
    # Gdp needed at part3 of the code
    # Load and clean
    gdp = pd.read_excel("../data/gdplev.xls", skiprows=5)
    gdp = gdp.rename(columns={"Unnamed: 0": "Year", 
                              "Unnamed: 4": "Year q", 
                              "GDP in billions of current dollars.1": "GDP current", 
                              "GDP in billions of chained 2009 dollars.1": "GDP chained 2009"})
    gdp.drop(labels=["Unnamed: 3", "Unnamed: 7"], axis = 1, inplace = True)
    columns = ["Year q", 
               "GDP current", 
               "GDP chained 2009"]
    gdp = gdp[columns]
    gdp = gdp.dropna()
    gdp = gdp.reset_index()
    del gdp["index"]
    
    # Load and clean City...
    ct = pd.read_csv("../data/city_zhvi_allhomes.csv")
    cols = list(ct.columns.values)    # List all columns
    q = ct[ct.columns[cols.index("2000-01"):len(ct)]] # Select columns after 2000 and save at dataframe "q"
    
    # Create quartes columns using index at gdp
    m = 0
    items = [items for items in gdp.loc[gdp[(gdp["Year q"] == "2000q1")].index[0]:len(gdp)]["Year q"]] # Columns to save
    for i in range(0,int((len(q.columns)-2)/3)): # Nº of quarters until 2016q2... data until 2016-08 -> 2016q3
        q[items[i]] = (q[q.columns[m]] + q[q.columns[m+1]] + q[q.columns[m+2]]) / 3 # New column with mean o 3 months: quarter
        m+=3
    q["2016q3"] = (q[q.columns[m]] + q[q.columns[m+1]]) / 2 # Data until 2016-08 -> 2016q3
    items.append("2016q3")
    q = q[items]
    
    ct = ct.replace(states) # Use dictionary to replace ct["State"] elements
    q = ct[["State", "RegionName"]].join(q[items], how="inner")
    q = q.set_index(["State", "RegionName"])
    
    return q

convert_housing_data_to_quarters()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [8]:
def run_ttest():
    """First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss)."""
    
    # Quarter recession starts: 2008q2
    # Quarter recession bottom 2009q2
    # price_ratio=quarter_before_recession/recession_bottom
    
    # Load convert_housing_data_to_quarters() and get_list_of_university_towns()
    q = convert_housing_data_to_quarters()
    df = get_list_of_university_towns()
    
    # Clean and modification
    q = q.sort_index()
    df["Uni"] = "U"
    df = df.set_index(["State", "RegionName"])
    todos = pd.merge(df, q, how="outer", left_index=True, right_index=True)
    todos["Ratio"] = todos["2008q3"] / todos["2009q2"]   # price_ratio=quarter_before_recession/recession_bottom
    tu = todos[todos["Uni"] == "U"]
    #tu = pd.merge(df, q, how="inner", left_index=True, right_index=True)
    tnu = todos[todos["Uni"] != "U"]
    columns = ["Uni", 
               "2008q2", 
               "2008q3", 
               "2008q4", 
               "2009q1",
               "2009q2", 
               "Ratio"]
    #todos[columns]
    test = ttest_ind(tu["Ratio"].dropna(), tnu["Ratio"].dropna())
    if test[1] <= 0.01:
        different = True # We reject the null hypothesis
    else:
        different = False
    
    if tu["Ratio"].dropna().mean() < tnu["Ratio"].dropna().mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return [different, test[1], better]

run_ttest()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[True, 0.005992806213991101, 'university town']